## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import pickle
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
# Prepare object points like (0,0,0), (1,0,0), (2,0,0), ...,(6,5,0)
objpts = np.zeros((6*9, 3), np.float32)
# Get the meshgrid points one by one and replace them in the x and y 
# coordinates of the object points
objpts[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

objpoints = []
imgpoints = []

def get_cam_params():
    images = glob.glob('camera_cal/*.jpg')

    for idx, frame_name in enumerate(images):
        image = cv2.imread(frame_name)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

        if ret == True:
            objpoints.append(objpts)
            imgpoints.append(corners)

            # Draw and display the corners
            cv2.drawChessboardCorners(image, (9,6), corners, ret)

            cv2.imshow('image', image)
            cv2.waitKey(1)
            
    test_img = cv2.imread('camera_cal/calibration4.jpg')
    imshape = test_img.shape

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (imshape[1], imshape[0]), None, None)

    result = cv2.undistort(test_img, mtx, dist, None, mtx)
    cv2.imwrite('camera_cal/test_undist.jpg', result)
    
    return mtx, dist
    
def store_cam_params(mtx, dist):
    # Acts like a dictionary that will be saved later using pickle
    pkl = {}
    pkl["mtx"] = mtx
    pkl["dist"] = dist
    pickle.dump( pkl, open( "camera_cal/camera_param_pickle.p", "wb" ) )
    cv2.destroyAllWindows()


In [3]:
# mtx, dist = get_cam_params()
# store_cam_params(mtx, dist)

In [50]:
def abs_sobel_thresh(img,  thresh=(0, 255), orient='x', sobel_kernel=5):
    '''
    Calculate directional gradient.
    '''

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    if orient == 'x':
        sobel_dir = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    else:
        sobel_dir = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    sobel_abs = np.absolute(sobel_dir)

    max_sobel = np.max(sobel_abs)
    scaled = np.uint8(255*(sobel_abs/max_sobel))

    grad_binary = np.zeros_like(scaled)
    grad_binary [(scaled > thresh[0]) & (scaled < thresh[1])] = 1

    return grad_binary

def mag_thresh(image, mag_thresh=(0, 255), sobel_kernel=3):
    '''Calculate gradient magnitude'''


    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Calculate the magnitude
    magnitude = np.sqrt((sobel_x ** 2) + (sobel_y ** 2))

    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled = np.uint8(magnitude * 255 / np.max(magnitude))
    # 5) Create a binary mask where mag thresholds are met
    mag_binary = np.zeros_like(scaled)
    mag_binary[(scaled > mag_thresh[0]) & (scaled < mag_thresh[1])] = 1
    # 6) Return this mask as your binary_output image

    return mag_binary

def dir_threshold(image, thresh=(0, np.pi/2), sobel_kernel=3):
    '''Calculate gradient direction'''

    # 1) Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    absx = np.absolute(sobelx)
    absy = np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient
    direction = np.arctan2(absy, absx)
    # 5) Create a binary mask where direction thresholds are met
    dir_binary = np.zeros_like(direction)
    dir_binary[(direction > thresh[0]) & (direction < thresh[1])] = 1
    # 6) Return this mask as your binary_output image

    return dir_binary

def hls_select(imag, thresh=(0, 1), channel='s'):

    hls = cv2.cvtColor(imag, cv2.COLOR_RGB2HLS)

    if channel == 'h':
        ch = 0
    elif channel == 'l':
        ch = 1
    else:
        ch = 2

    layer = hls[:, :, ch]

    # 3) Return a binary image of threshold result
    mask = np.zeros_like(layer)
    # mask = mask * 200

    mask[(layer >= thresh[0]) & (layer <= thresh[1])] = 1

    # porcarie(thresh, channel)

    # print(channel, thresh[0], thresh[1], np.where(layer<1)[1].shape)
    # print(np.average(mask), np.max(layer))

    return mask

def lab_select(image, thresh=(0, 255), channel='l'):

#     image = image.astype(np.float32)
#     image = image/255
    
    cie_lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)

    if channel == 'l':
        ch = 0
        channel = "CIE L"
    elif channel == 'a':
        ch = 1
        channel = "CIE A"
    elif channel == 'b':
        ch = 2
        channel = "CIE B"

    layer = cie_lab[:, :, ch]
    

#     print(layer.dtype)
#     print(np.max(layer))

    if channel == 'CIE L':
#         print("Recalculated thresholds")
        
        a = 39.20193
        b= 5.121302
        c = 42.48095
        d = 88.16211

        avg = np.average(layer[420:,:])
        
        # y = 88.16211 + (39.20193 - 88.16211) / (1 + (x / 42.48095) ^ 5.121302)
        
        thresh0 = d + (a - d) / (1 + (avg / c) ** b)
        thresh0 = thresh0
        thresh1 = 100
#         print(channel, "\t", avg, "\t thres:", thresh0)

    else:
        thresh0 = thresh[0]
        thresh1 = thresh[1]
        
#     print(channel)
#     print(np.min(layer))

    # 3) Return a binary image of threshold result
    mask = np.zeros_like(layer)
    
    mask[(layer >= thresh0) & (layer <= thresh1)] = 1
    
#     cv2.imshow('img', mask)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()


    return mask


In [51]:
def combined_image(image):
    
    sobel_low = 20
    sobel_high = 255
    mag_low = 40
    mag_high = 255
    
    white_low = 8
    white_high = 86
    yellow_low = 3
    yellow_high = 255
    
    lum_low = 0.72 * 255
    lum_high = 1 * 255
    sat_low = 0.26 * 255
    sat_high = 1 * 244
    
    diam = 9
    sig_col = 13
    sig_sp = 31

    
    blur_filter = cv2.bilateralFilter(image, d=diam, sigmaColor=sig_col, sigmaSpace=sig_sp)

    cie_l = lab_select(blur_filter)
    cie_b = lab_select(blur_filter, (18, 100), "b")
    
    combined = np.zeros_like(cie_l)
    
    combined[(cie_l == 1) | (cie_b == 1)] = 1
    
    return np.dstack((combined, combined*0, combined*0))



In [52]:
def process_images(image):
    
    # Convert image from float32 to uint8
    image = image.astype(np.float32)
    image = image/255

        
    # Send the image for processing to combined_image()
    comb = combined_image(image)

    # Convert the image back to uint8
    weighted = cv2.addWeighted(image, 0.5, comb, 1, 0)
    uint_img = cv2.convertScaleAbs(weighted * 255)
    
#     uint_img = cv2.cvtColor(uint_img, cv2.COLOR_BGR2RGB)
    
    
    return uint_img

    
'''problem_frames/9.png'''

img = cv2.imread('problem_frames/14.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
proc = process_images(img)
    
cv2.imshow('img', proc)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [53]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [54]:
white_output = 'test_videos_output/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4").subclip(20,25)
# clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_images) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   2%|▏         | 3/125 [00:00<00:07, 15.75it/s, now=None]

Moviepy - Building video test_videos_output/project_video.mp4.
Moviepy - Writing video test_videos_output/project_video.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/project_video.mp4
CPU times: user 44 s, sys: 1.14 s, total: 45.1 s
Wall time: 12.7 s


In [55]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))